In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
import pandas as pd
import numpy as np
import torch

import json
from pathlib import Path
from copy import deepcopy

import pyciemss.visuals.vega as vega
from pyciemss.utils import get_tspan
from pyciemss.utils.interface_utils import convert_to_output_format


In [33]:
# Load cached data...
tspan = get_tspan(1, 50, 500)

root = Path("..")/"test"/"data/"
prior_samples = convert_to_output_format(vega.tensor_load(root/"prior_samples.json"), tspan)
intervened_samples = vega.tensor_load(root/"intervened_samples.json")
observed_trajectory = convert_to_output_format(vega.tensor_load(root/"observed_trajectory.json"), tspan)

observed_points = (observed_trajectory
    .rename(columns={"Rabbits_sol": "Rabbits Example"})
    .drop(columns=["Wolves_sol", "alpha_param", "beta_param", "delta_param", "gamma_param"])
    .iloc[::10]
)


In [81]:
observed_points.set_index(["timepoint_id", "sample_id"]).melt(ignore_index=False, var_name="trajectory").reset_index().to_dict(orient="records")

[{'timepoint_id': 0,
  'sample_id': 0,
  'trajectory': 'Rabbits Example',
  'value': 0.5774485468864441},
 {'timepoint_id': 10,
  'sample_id': 0,
  'trajectory': 'Rabbits Example',
  'value': 0.4931105971336365},
 {'timepoint_id': 20,
  'sample_id': 0,
  'trajectory': 'Rabbits Example',
  'value': 0.5761047005653381},
 {'timepoint_id': 30,
  'sample_id': 0,
  'trajectory': 'Rabbits Example',
  'value': 0.7969658970832825},
 {'timepoint_id': 40,
  'sample_id': 0,
  'trajectory': 'Rabbits Example',
  'value': 1.1683192253112793},
 {'timepoint_id': 50,
  'sample_id': 0,
  'trajectory': 'Rabbits Example',
  'value': 1.6221495866775513},
 {'timepoint_id': 60,
  'sample_id': 0,
  'trajectory': 'Rabbits Example',
  'value': 1.7182937860488892},
 {'timepoint_id': 70,
  'sample_id': 0,
  'trajectory': 'Rabbits Example',
  'value': 1.0860116481781006},
 {'timepoint_id': 80,
  'sample_id': 0,
  'trajectory': 'Rabbits Example',
  'value': 0.6113320589065552},
 {'timepoint_id': 90,
  'sample_id': 0

In [45]:
observations = prior_samples.set_index(["timepoint_id", "sample_id"]).drop(columns=["alpha_param", "beta_param", "delta_param", "gamma_param"])
long = observations.melt(ignore_index=False, var_name="trajectory").set_index("trajectory", append=True)
long.head()

,,,value
timepoint_id,sample_id,trajectory,
0,0,Rabbits_sol,0.485634
1,0,Rabbits_sol,0.461909
2,0,Rabbits_sol,0.441513
3,0,Rabbits_sol,0.424107
4,0,Rabbits_sol,0.409390


In [82]:
nice_labels={"Rabbits_sol": "Rabbits", "Wolves_sol": "Wolves"}

schema = vega.trajectories(prior_samples, 
                           subset=".*_sol", 
                           relabel=nice_labels,
                           points=observed_points
                           )
schema = vega.pad(schema, 5)
vega.save_schema(schema, "_schema.json")
vega.ipy_display(schema)

In [84]:
nice_labels={"Rabbits_sol": "Rabbits", "Wolves_sol": "Wolves"}
schema = vega.trajectories(prior_samples, 
                           subset=".*_sol", 
                           relabel=nice_labels,
                           points=observed_points,
                           colors={"Rabbits_sol": "black", "Wolves_sol": "red"})
schema = vega.pad(schema, 5)
vega.save_schema(schema, "_schema.json")
vega.ipy_display(schema)

In [86]:
ext_samples = deepcopy(prior_samples)
ext_samples["Wolf Example"] = prior_samples["Wolves_sol"][1]
ext_samples["Rabbit Example"] = prior_samples["Rabbits_sol"][1]
subset = ["Rabbits_sol", "Wolves_sol", "Rabbit Example", "Wolf Example"]
nice_labels={"Rabbits_sol": "Rabbits", "Wolves_sol": "Wolves"}
schema = vega.trajectories(ext_samples, 
                           subset=subset, 
                           relabel=nice_labels)
schema = vega.pad(schema, 5)
vega.save_schema(schema, "_schema.json")
vega.ipy_display(schema)
#tracks = schema